In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wave as w
import copy
np.set_printoptions(precision=2)
from cycler import cycler
plt.rcParams.update(plt.rcParamsDefault)
w.Spectrum.k = np.linspace(2 * np.pi / (1.0 * w.MICRO), 2 * np.pi / (300.0 * w.NANO), 1000)

# Example usage of simulated materials
This notebook is ment as an reference on how to use `Spectrum` and `Material` classes,
and for visual test of the implementation.

### Generate an example spectrum

In [ ]:
s = w.GaussianSpectrum(mean=w.GREEN, std=(5*w.MICRO))
# s = w.DeltaSpectrum(wavelength=w.GREEN)
z = np.linspace(-6*w.MICRO, 6*w.MICRO, 1000)
s.delay(0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
s.plot(ax=ax1, spectrum_axis=ax1.twinx(), wavelength=True)
ax1.legend()
s.plot_amplitude(z, ax=ax2)
ax2.legend()
fig.tight_layout() 
plt.show()

### Compare different materials

#### Analitical vs constant vs matrix theory

In [ ]:
air = w.EmptySpace(z=z, name="analytic")
common_n = 1
fixed_dielectric = w.ConstantMaterial(z=z, n0=common_n, name="phase shift")
dielectric = w.SimpleDielectric(z=z, n0=1, name="transfer matrices")

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

air.plot(ax1, imaginary_axis=ax1.twinx())
fixed_dielectric.plot(ax2, imaginary_axis=ax2.twinx())
dielectric.plot(ax3, imaginary_axis=ax3.twinx())
fig.tight_layout() 
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

air.record(s,s)
air.plot_recent_energy(ax1, label="air")
dielectric.record(s,s)
dielectric.plot_recent_energy(ax1, label="matrix theory")
fixed_dielectric.record(s,s)
fixed_dielectric.plot_recent_energy(ax1, label="correlation")
ax1.legend()

air.transmit(s).plot(ax2)
dielectric.transmit(s).plot(ax2, spectrum_axis=ax2.twinx())
fixed_dielectric.transmit(s).plot(ax2, label="reflected")
ax2.legend()
plt.show()

### Layered material

In [ ]:
z_layered = np.linspace(-2*w.MICRO, 2*w.MICRO, 1000)
layered = w.LayeredMaterial(z=z_layered, n_layers=20, layer_width=80*w.NANO, n0=1.5, n1=2.0)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

layered.shade_plot(ax1, alpha=0.2)
layered.plot(ax1, imaginary_axis=ax1.twinx())
layered.shade_plot(ax2, alpha=0.2)
layered.reflect(s).plot(ax3, label="reflected", wavelength=True, spectrum_axis=ax3.twinx())
ax3.legend()
layered.record(s, s)
layered.plot_recent_energy(ax2)
fig.tight_layout() 
plt.savefig("layered2.pdf")
plt.show()

### An example recording

In [ ]:
z = np.linspace(-6*w.MICRO, 6*w.MICRO, 1000)
dielectric = w.SimpleDielectric(z=z, n0=1.45, name="transfer matrices")
s0 = w.GaussianSpectrum(mean=w.GREEN, std=(5*w.MICRO))
dielectric.record(s0, s0)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
dielectric.plot_recent_energy(ax1)
ax1.set_title("interference")
dielectric.plot(imaginary_axis=ax2.twinx(), ax=ax2)
dielectric.reflect(s0).plot(ax3, spectrum_axis=ax3.twinx())
plt.tight_layout()
plt.show()

In [ ]:
steps = 20
plt.rcParams['axes.prop_cycle'] = cycler('color', plt.get_cmap('viridis')(np.linspace(0, 1, int(steps/10)+1)))

In [ ]:
dielectric = w.SimpleDielectric(z=z, n0=1)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
ax3twin = ax3.twinx()
ax2twin = ax2.twinx()

for pulse in range(steps):
    phase = 0
    s2 = s0 * np.exp(1j*phase)
    if pulse % 10 == 9:
        dielectric.plot(ax=ax2, imaginary_axis=ax2twin, alpha=0.5)
    dielectric.record(s0, s2)
    if pulse % 10 == 9:
        ax1.plot(dielectric.z[:-1], dielectric.recent_energy[:-1], label="pulse {}".format(pulse + 1), alpha=0.5)
        dielectric.reflect(s0).plot(ax3, spectrum_axis=ax3twin)
        
plt.tight_layout()        
plt.show()